run on virtual_env

    Data Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime

In [2]:
df_train = pd.read_csv("data/train.csv")
df_train = df_train.drop(columns=["id"])
print(df_train.shape)
df_train.head()

(3000888, 5)


,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


In [3]:
df_stores = pd.read_csv("data/stores.csv")
print(df_stores.shape)
df_stores.head()

(54, 5)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [4]:
df_holidays = pd.read_csv("data/holidays_events.csv")
df_holidays["date"] = pd.to_datetime(df_holidays["date"])
print(df_holidays.shape)
print(df_holidays.type.unique())
df_holidays.head()

(350, 6)
['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event']


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [5]:
# Oil is an important economic indicator for countries like ecuador
df_oil = pd.read_csv("data/oil.csv")
print(df_oil.shape)
df_oil.head()

(1218, 2)


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
# Fill in null values of df_oil by interpolating the values
# And make the first row equal to the next one with bfill
df_oil = df_oil.interpolate().bfill()
df_oil.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [7]:
df_transactions = pd.read_csv("data/transactions.csv")
df_transactions["date"] =  pd.to_datetime(df_transactions["date"])
print(df_transactions.shape)
df_transactions.head()

(83488, 3)


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


    Merge

In [8]:
# Join df_train and df_stores
df = pd.merge(df_train, df_stores, on="store_nbr", how="left")
df["date"] = pd.to_datetime(df["date"])
df

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6
3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6
3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6


In [9]:
# Build a function that handles the holidays
# Get the row from the df you wanna assign the holidays to
# And get the holidays df
def get_holiday_info(row, holidays):

    # Filter the holidays df by date of the row of the main df
    # If holiday is transferred to another day, then that day is not a holiday, so only take the non-transferred holidays
    # The location of the holiday is also important
    # Get the day only if it's in the same country or the same region of the same city of the store
    tmp = holidays[(holidays["date"] == row["date"]) & (holidays["transferred"] == False) &
                   (
                       (holidays["locale"] == "National") |
                       ((holidays["locale"] == "Regional") & (holidays["locale_name"] == row["state"])) | 
                       ((holidays["locale"] == "Local") & (holidays["locale_name"] == row["city"]))
                   )
                   ]

    # tmp is not-empty if there is a holiday in that day in the city of the store
    # Sometimes that day is work day because it says so in the holidays_df
    if len(tmp) > 0:
        #print(tmp.iloc[0])
        tmp = tmp.reset_index().iloc[0]
        if tmp["type"] == "Work Day":
            return pd.Series({'isHoliday': 0, 'reason': "Work Day"})
        else:
            return pd.Series({'isHoliday': 1, 'reason': "Holiday"})
    
    # if tmp is empty, that day is not a holiday unless it's weekend
    else:
        #print("Not a holiday:", row["date"])
        #print("Day num:", row["date"].weekday())
        #input_date = datetime.datetime.strptime(tmp["date"], '%Y-%m-%d')
        if row["date"].weekday() in [5, 6]:
            return pd.Series({'isHoliday': 1, 'reason': "Weekend"})
        else:
            return pd.Series({'isHoliday': 0, 'reason': "Week Day"})
        

In [10]:
# Apply the function
df_n = df[["date", "city", "state"]].drop_duplicates()
new_cols = df_n.apply(get_holiday_info, holidays=df_holidays, axis=1)
df_n = pd.concat([df_n, new_cols], axis=1)
df_n

,date,city,state,isHoliday,reason
0,2013-01-01,Quito,Pichincha,1,Holiday
66,2013-01-01,Cayambe,Pichincha,1,Holiday
99,2013-01-01,Latacunga,Cotopaxi,1,Holiday
165,2013-01-01,Riobamba,Chimborazo,1,Holiday
198,2013-01-01,Ibarra,Imbabura,1,Holiday
...,...,...,...,...,...
3000129,2017-08-15,Loja,Loja,0,Week Day
3000228,2017-08-15,Machala,El Oro,0,Week Day
3000327,2017-08-15,Esmeraldas,Esmeraldas,0,Week Day
3000657,2017-08-15,Manta,Manabi,0,Week Day


In [12]:
# Add the holidays info to the df
df = pd.merge(df, df_n, on=["date", "city", "state"], how="left")

In [13]:
# Add the transactions info to the df
df = pd.merge(df, df_transactions, on=["date", "store_nbr"], how="left")

In [19]:
# Reorder columns
cols = ['date', 'state', 'city', 'isHoliday', 'reason', 'store_nbr', 'type', 'cluster', 'transactions', 'family',  'onpromotion', 'sales']
df = df[cols]
print(df.shape)
df.head()

(3000888, 12)


,date,state,city,isHoliday,reason,store_nbr,type,cluster,transactions,family,onpromotion,sales
0,2013-01-01,Pichincha,Quito,1,Holiday,1,D,13,NaN,AUTOMOTIVE,0,0.0
1,2013-01-01,Pichincha,Quito,1,Holiday,1,D,13,NaN,BABY CARE,0,0.0
2,2013-01-01,Pichincha,Quito,1,Holiday,1,D,13,NaN,BEAUTY,0,0.0
3,2013-01-01,Pichincha,Quito,1,Holiday,1,D,13,NaN,BEVERAGES,0,0.0
4,2013-01-01,Pichincha,Quito,1,Holiday,1,D,13,NaN,BOOKS,0,0.0


    Data Prep

In [20]:
# Remove the first date of the dataset as there are many data loss this day
df = df[df["date"]>pd.to_datetime("2013-01-01")].reset_index(drop=True)
df.head()

,date,state,city,isHoliday,reason,store_nbr,type,cluster,transactions,family,onpromotion,sales
0,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,AUTOMOTIVE,0,2.0
1,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BABY CARE,0,0.0
2,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BEAUTY,0,2.0
3,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BEVERAGES,0,1091.0
4,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BOOKS,0,0.0


In [21]:
# Some stores probably opened after the beginning of date column
# So we have to remove the rows for each store before their opening day
# If sales = 0 for a long time starting from the first date, we remove all the rows until the first non zero value for that store
df_empty = pd.DataFrame()
for i in df["store_nbr"].sort_values().unique():
    df_temp = df[df["store_nbr"]==i].reset_index(drop=True)
    df_temp2 = df_temp[["date","store_nbr","sales"]].groupby(by=["date","store_nbr"]).sum().reset_index()
    first_purchase_date = df_temp2[df_temp2['sales'] != 0].date.iloc[0]
    df_temp = df_temp[df_temp.date >= first_purchase_date]
    df_empty = pd.concat([df_empty, df_temp])
df = df_empty.copy()

In [22]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2778798 entries, 0 to 55538
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   date          2778798 non-null  datetime64[ns]
 1   state         2778798 non-null  object        
 2   city          2778798 non-null  object        
 3   isHoliday     2778798 non-null  int64         
 4   reason        2778798 non-null  object        
 5   store_nbr     2778798 non-null  int64         
 6   type          2778798 non-null  object        
 7   cluster       2778798 non-null  int64         
 8   transactions  2755071 non-null  float64       
 9   family        2778798 non-null  object        
 10  onpromotion   2778798 non-null  int64         
 11  sales         2778798 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 275.6+ MB


In [23]:
# Transactions has null-values. Let's observe
df_temp = df.loc[df["transactions"].isna()==True,["date","store_nbr","sales"]].groupby(by=["date","store_nbr"]).sum().reset_index().sort_values(by=["store_nbr","date"])
#df_temp = pd.merge(df_temp, df["date","store_nbr","isHoliday"], on=["date", "store_nbr"], how="left")
df_temp#[df_temp["sales"]==0]

,date,store_nbr,sales
27,2014-01-01,1,0.000000
213,2015-01-01,1,0.000000
320,2015-07-07,1,0.000000
321,2016-01-01,1,0.000000
374,2016-01-02,1,8877.175000
...,...,...,...
259,2015-01-01,54,0.000000
373,2016-01-01,54,0.000000
443,2016-01-03,54,11774.518000
482,2016-01-04,54,8711.512998


There are some null values of transaction dataset. One would think that, if transactions == Null then there is no sales that day. <br>
df_temp shows us that this is not true. <br>
For now, we keep transactions but later we might drop it also because of the fact that it does not exist in test.csv

    Write the df into a pickle file

In [24]:
df

,date,state,city,isHoliday,reason,store_nbr,type,cluster,transactions,family,onpromotion,sales
0,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,AUTOMOTIVE,0,2.000
1,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BABY CARE,0,0.000
2,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BEAUTY,0,2.000
3,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BEVERAGES,0,1091.000
4,2013-01-02,Pichincha,Quito,0,Week Day,1,D,13,2111.0,BOOKS,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
55534,2017-08-15,Manabi,El Carmen,0,Week Day,54,C,3,802.0,POULTRY,0,59.619
55535,2017-08-15,Manabi,El Carmen,0,Week Day,54,C,3,802.0,PREPARED FOODS,0,94.000
55536,2017-08-15,Manabi,El Carmen,0,Week Day,54,C,3,802.0,PRODUCE,76,915.371
55537,2017-08-15,Manabi,El Carmen,0,Week Day,54,C,3,802.0,SCHOOL AND OFFICE SUPPLIES,0,0.000


In [25]:
df.to_pickle("data/data_prepped.pkl")

People working in public sector usually get their salaries on 15th of each month and at the end of the month. Label these days

16 Apr 2016 -> An earthquake of 7.8. People bought a lot of food and donated them. So, we can check the sales of those days.